# Travelling Personal Assistant

#### Initialize OpenAI API Clients

In [ ]:
import os
import openai
import IPython
import ipywidgets as widgets

from IPython.display import display, Markdown

os.environ['OPENAI_API_KEY'] = 'sk-proj-t5af7zZmWhB5TxQ6wT5xT3BlbkFJOKJuzSx8ZuFqpvZbmXbx'
openai.api_key = os.environ.get('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()

#### Generate Travel Itinerary Using Chat Completion

In [ ]:
def generate_travel_itinerary(destination, days=7, model='gpt-3.5-turbo', temperature=1):
    
    system_message = """
    You are a travel planning assistant. Your goal is to make the travel planning experience easy, faster, and efficient for travelers. You will help users plan their trips by providing comprehensive travel itineraries, destination recommendations, transportation options, accommodation suggestions, packing lists, and local tips. Consider all aspects of travel, including flights, trains, car rentals, hotels, activities, dining options, cultural attractions, and safety advice. Your responses should be friendly, informative, and tailored to the specific needs of the traveler.
    """
    
    user_prompt = f"""
    ## Setup:
    - persona: Friendly, knowledgeable, and efficient travel planning assistant.
    - context: The user is planning a {days}-day trip to {destination} and needs a detailed travel itinerary.

    ## The Instruction:
    - instruction: Create a detailed {days}-day travel itinerary for a trip to {destination}, including recommendations for flights, accommodations, local transportation, must-see attractions, dining options, cultural tips, and safety advice. Ensure the itinerary is easy to follow and covers a variety of experiences, from sightseeing to local cuisine.
    - few-shot prompting: 
      - Day 1: Arrival in Tokyo
        - Morning: Arrive in Tokyo and check into your hotel.
        - Afternoon: Explore Asakusa and visit the Sensoji Temple.
        - Evening: Dinner at a local sushi restaurant.
      - Day 2: Tokyo
        - Morning: Visit the Tsukiji Fish Market.
        - Afternoon: Walk through the Ginza shopping district.
        - Evening: Enjoy a night view from Tokyo Tower.
    - chain of thought: Plan each day, considering transportation and accommodation, recommending attractions and dining options, and adding cultural tips and safety advice.

    ## The Output:
    - The output should be based on the chain of thought for the unique destination the user inputted.
    - format: Structured in a clear, day-by-day itinerary format.
    - additional restriction: Ensure the itinerary includes diverse experiences, from sightseeing and local cuisine to cultural tips and safety advice.
    <Format: format everything in text.>
    """
    
    response = client.chat.completions.create( 
        model=model,
        messages=[
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': user_prompt}
        ],
        temperature=temperature,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return response.choices[0].message.content

#### Ask the user's destination

In [ ]:
place = input('Hello, I am your Personal Travel Planning Assistant, May I know where you want to go? ')
itinerary = generate_travel_itinerary(destination=place)
display(Markdown(itinerary))

#### Flowchats of the Travel Plans Using DALL-E

In [ ]:
# Step 3: Visualize Destinations Using DALL-E
def visualize_destination(itinerary, model='dall-e-3', size='1792x1024', quality='standard'):
    image_prompt = f'Generate a flowcharts that visualizes the provided 7-day travel {itinerary}'
    image_response = client.images.generate(
        model=model,
        prompt=image_prompt,
        style='natural',
        size=size,  # 1024x1024, 1024x1792, 1792x1024 pixels
        quality=quality
    )
    
    
    image_url = image_response.data[0].url
    display(widgets.HTML(f"<img src='{image_url}' width='400'>"))


In [ ]:
# Visualize the flowchat generated using DALL-E-3
visualize_destination(itinerary)

#### Voice commands for Travel Planning Using TTS (Text To Speech)Model 

#### The AI Assistant will read aloud the travel itinerary to you if you dont have time to read it. 

In [ ]:
def generate_tts(itinerary):
    introduction = "I am your Personal Travel Assistant, I will read out your travel plan. Here is your itinerary:\n\n"
    end_word = "I am always here to here you out all through your trips. Thank you\n\n"
    tts_text = introduction + itinerary + end_word
    return tts_text

input  = generate_tts(itinerary)

response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input= input
)

response.stream_to_file("speech.mp3")
IPython.display.Audio("speech.mp3")

#### Ask your Personal Travel Planning Assistant anything regarding your travel plans

In [ ]:
client = OpenAI()

def generate_response(user_prompt, system_role = f'You are a travel planning assistant. Study the content of {itinerary} to use any question the user asked you. In the itinerary "Day" is word as delimiter to know the day number and content. Your goal is to assist the user based on the provided itinerary for their trip. Answer the user question and assist them in an engaging and conversiional manner. You will answer questions and provide information only related to the details in the itinerary. Ensure your responses are clear, informative, and based solely on the itinerary content. Always refer to the specific day and activities listed in the itinerary.',
                      model='gpt-3.5-turbo', temperature = 1 ):
    messages=[
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_prompt}
    ]
    
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=temperature)
    
    return response.choices[0].message.content


#### Ask any question here

In [ ]:
response_input = "What am I going to do in the fifth day?"

In [ ]:
response = generate_response(response_input)
display(Markdown(response))

#### AI will answer your question in voice

In [ ]:
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input= response
)

response.stream_to_file("speech.mp3")
IPython.display.Audio("speech.mp3")